# Part I. Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import pandas as pd
import time
from tqdm import tqdm

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821



**we can now work with `event_datafile_new.csv`, located within the Workspace directory.**  

The `event_datafile_new.csv` contains the following columns:
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

# Part II. ETL for Cassandra #

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

In [8]:
# Set pandas_factory
# Thanks to ragesz 
# source https://stackoverflow.com/a/41484806
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

----

### Create queries to ask the following three questions of the data ###

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




_____

#### Define functions ####

****Function Name:create_table****
* Parameters: create query AS String
* Returns: None
* Dependency: pandas_factory
* Notes: creates a table

****Function Name:add_to_table****
* Parameters: insert query AS String, columns names As Array, Optional csv_file path
* Returns: None
* Dependency: None
* Notes: Lodes data into a table

****Function Name:excute_find****
* Parameters: find query AS String
* Returns: search results as pandas dataframe
* Dependency: pandas_factory
* Notes: search table for given query and results as dataframe

In [9]:
def create_table(query):
    try:
        print("Creating the tables")
        session.row_factory = pandas_factory
        session.default_fetch_size = None #needed for large queries, otherwise driver will do pagination.
        session.execute(query)
    except Exception as e:
        print(e)        

In [10]:
def add_to_table(query, columns, csv_file = 'event_datafile_new.csv'):
    try:
        tdf = pd.read_csv(csv_file)
    except Exception as e:
        print(e) 
    print("Loading data into table")
    time.sleep(0.1)
    for row in tqdm(range(len(tdf))):
        queryData = tdf[columns].values[row].tolist()
        session.execute(query, queryData)
        

In [11]:
def excute_find(query):
    try:
        print("Running query")
        rows = session.execute(query)
        return rows._current_rows #Used Panda dataframes to add columns to your query output
    except Exception as e:
        print(e)

----

## Query 1 ##
* `Return:` artist, song title and song's length
* `Where:` sessionId = 338, and itemInSession  = 4

#### Define variables ####

In [12]:
#Define tables
song_info_by_sessionid_create = """
        CREATE TABLE IF NOT EXISTS song_info_by_sessionid (
            session_id int,
            item_in_session int,
            artist text,
            song_title text,
            song_length float,
            PRIMARY KEY(session_id, item_in_session)
        );
        """

#Define inserts
song_info_by_sessionid_insert = """
        INSERT INTO song_info_by_sessionid
            (session_id, item_in_session, artist, song_title, song_length) 
            VALUES(%s, %s, %s, %s, %s)
        """

#Define columns to be selected
song_info_by_sessionid_columns = ['sessionId', 'itemInSession', 'artist', 'song', 'length']


#Define query
song_info_by_sessionid_query = """
        SELECT artist, song_title, song_length 
        FROM song_info_by_sessionid
        WHERE session_id = 338 AND item_in_session = 4;
        """

#### Execute Query #### 

In [13]:
create_table(song_info_by_sessionid_create)
add_to_table(song_info_by_sessionid_insert, song_info_by_sessionid_columns)
song_info_by_sessionid_return = excute_find(song_info_by_sessionid_query)

  0%|          | 0/6820 [00:00<?, ?it/s]

Creating the tables
Loading data into table


100%|██████████| 6820/6820 [00:28<00:00, 236.48it/s]

Running query


#### Print Results ####

In [15]:
song_info_by_sessionid_return

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


----

## Query 2##
* `Return:` name of artist, song title (sorted by itemInSession) and user (first and last name)
* `Where:` userid = 10, sessionid = 182

#### Define variables ####

In [16]:
#Define tables
song_info_by_user_create = """
        CREATE TABLE IF NOT EXISTS song_info_by_user (
            user_id int,
            session_id int,
            item_in_session int,
            user_first_name text,
            user_last_name text,
            artist text,
            song_title text,
            PRIMARY KEY ((user_id, session_id), item_in_session)
        );
    """

#Define inserts
song_info_by_user_insert = """
        INSERT INTO song_info_by_user
        (user_id, session_id, item_in_session, user_first_name, user_last_name, artist, song_title) 
        VALUES(%s, %s, %s, %s, %s, %s, %s)
        """

#Define columns to be selected
song_info_by_user_columns = ['userId', 'sessionId', 'itemInSession', 'firstName', 'lastName', 'artist', 'song']

#Define query
song_info_by_user_query = """
        SELECT artist, song_title, user_first_name, user_last_name 
        FROM song_info_by_user
        WHERE user_id = 10 AND session_id = 182;
    """

#### Execute Query ####

In [17]:
create_table(song_info_by_user_create)
add_to_table(song_info_by_user_insert, song_info_by_user_columns)
song_info_by_user_return = excute_find(song_info_by_user_query)                      

  0%|          | 0/6820 [00:00<?, ?it/s]

Creating the tables
Loading data into table


100%|██████████| 6820/6820 [00:30<00:00, 224.81it/s]

Running query


#### Print Results ####

In [18]:
#Print dataframe
song_info_by_user_return

,artist,song_title,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


----

## Query 2##
* `Return:` all user name (first and last)
* `Where:` WHERE song_title = 'All Hands Against His Own'

#### Define variables ####

In [19]:

#Define tables
user_info_by_song_create = """
        CREATE TABLE IF NOT EXISTS user_info_by_song (
            song_title text,
            user_id int,
            user_first_name text,
            user_last_name text,
            PRIMARY KEY(song_title, user_id)
        );
        """

#Define insert
user_info_by_song_insert ="""
        INSERT INTO user_info_by_song
        (song_title, user_id, user_first_name, user_last_name) 
        VALUES(%s, %s, %s, %s)
        """

#Define columns to be selected
user_info_by_song_columns = ['song', 'userId', 'firstName', 'lastName']

#Define query
user_info_by_song_query = """
        SELECT user_first_name, user_last_name 
        FROM user_info_by_song
        WHERE song_title = 'All Hands Against His Own';
        """

#### Execute Query ####

In [20]:
create_table(user_info_by_song_create)
add_to_table(user_info_by_song_insert, user_info_by_song_columns)
user_info_by_song_return = excute_find(user_info_by_song_query) 

Creating the tables
Loading data into table


100%|██████████| 6820/6820 [00:24<00:00, 290.97it/s]

Running query


#### Print Results ####

In [21]:
user_info_by_song_return

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


----

## Clean-up ##

#### Drop the tables before closing out the sessions ####

In [22]:
## Drop the table before closing out the sessions
try:
    session.execute("DROP TABLE IF EXISTS song_info_by_sessionid;")
    session.execute("DROP TABLE IF EXISTS song_info_by_user;")
    session.execute("DROP TABLE IF EXISTS user_info_by_song;")
except Exception as e:
    print(e)

#### Close the session and cluster connection ####

In [23]:
session.shutdown()
cluster.shutdown()

----